# Introduction


The different bfabricPy packages are currently maintained in a single mono-repo at [github.com/fgcz/bfabricPy](https://github.com/fgcz/bfabricPy).

You can find changelogs and the updated documentation there.

## Installation

`bfabric-cli` is available as part of the [bfabric-scripts PyPI package](https://pypi.org/project/bfabric-scripts/) and can be installed using your Python package manager of choice.

To avoid issues on deployments on machines with old Python versions, we recommend using a tool like `uv` to install the package,
as it manages Python for you and will ensure that users have an appropriate version of Python installed:

```bash
uv tool install -p 3.13 bfabric-scripts
```

Older versions may be supported as well (as of 2025-02-25 the oldest supported version is Python 3.11).

## Configuration

To configure bfabricPy you need to obtain your webservicpassword and create a file `~/.bfabricpy.yml` with the following structure:

```yaml
GENERAL:
  # Normally, we would write PRODUCTION here, but,
  # for the sake of the tutorial we are defaulting to TEST as a fail-safe guard.
  default_config: TEST

PRODUCTION:
  login: yourBfabricLogin
  password: yourBfabricWebPassword
  base_url: https://fgcz-bfabric.uzh.ch/bfabric

TEST:
  login: yourBfabricLogin
  password: yourBfabricWebPassword
  base_url: https://fgcz-bfabric-test.uzh.ch/bfabric
```

You can add more configurations, e.g. if you have a system user that you want to use only in some scenarios.

In the rest of this tutorial, always make sure to set the `BFABRICPY_CONFIG_ENV=TEST` environment variable, as follows:

```bash
export BFABRICPY_CONFIG_ENV=TEST
```

Because this is a jupyter notebook, I also make sure to set it for the current Python process:

In [ ]:
import os
os.environ["BFABRICPY_CONFIG_ENV"] = "TEST"

## Getting Help
The interface of bfabric-cli is currently being developed, however you can explore the commands by calling `--help` on an arbitrary subcommand,
for instance:

In [ ]:
!bfabric-cli --help

The commands are organized in a hierarchy, and you can call `--help` on any subcommand in the tree to check if it has further subcommands, or, 
what its parameters are.

While the documentation of parameters is comprehensive the only current limitation is that we do not get a nice "usage" line showing
exactly how the parameters are supposed to be used.

In general, if possible it makes sense to use the parameters with their name, as not all parameters may remain available forever.
How to version/stabilize these CLI APIs is still an open question.

In [ ]:
!bfabric-cli api create --help

Please note that this documentation is autogenerated from the code using [cyclopts](https://github.com/BrianPugh/cyclopts)
and you should ideally provide as many parameters by name rather than position. Especially sequences are not really clearly documented, which hopefully will be resolved later.

# Generic entity API

## Reading entities
Generic functionality to read entities is available in `bfabric-cli api read`.

In [ ]:
!bfabric-cli api read --help

As you can see, we first need to provide the endpoint, and a query, we can specify the format as well there are some additional fields.

```bash
bfabric-cli api read resource --limit 10
```

The default table is not rendering correctly in Jupyter as of now (2025-02-25), so we execute this in a shell.

We can control the columns displayed:

```bash
bfabric-cli api read resource --limit 10 --columns id,name,relativepath,description,filechecksum
```

In addition to the interactive table format, we also support `json` and `yaml` outputs.
For instance to get the results in JSON format (restricting the columns to the same subset as before):

```bash
bfabric-cli api read resource --limit 10 --columns id,name,relativepath,description,filechecksum --format json
```

For instance to read all resources created by `pfeeder` between 2024-05-01 and 2024-05-02 (exclusive) we can execute the following command.

```bash
bfabric-cli api read resource createdby pfeeder createdafter 2024-05-01 createdbefore 2024-05-02 --columns id,relativepath,createdby
```

A small gimmick, is that it also prints the bfabricPy query, e.g. in this case we can perform the same operation in Python:

In [ ]:
from bfabric import Bfabric
client = Bfabric.from_config()
results = client.read(endpoint='resource', 
    obj={'createdby': 'pfeeder', 'createdafter': '2024-05-01', 'createdbefore': '2024-05-02'}, max_results=100)
results[0]

One useful feature, is being able to supply multiple values for the same key.  
The way this is done is you repeat the key-value pair for each additional value.

```bash
bfabric-cli api read resource id 2784586 id 2784576 id 2784573
```

Everything but the main output will be printed to standard error, so it's easier to pipe the main output.
However, if you want to persist the output to a file it's recommended to use the `--file` argument instead.

In [ ]:
!bfabric-cli api read resource createdby pfeeder createdafter 2024-05-01 createdbefore 2024-05-02 \
    --format yaml --file result.yml 

In [ ]:
!ls -lh result.yml

## Creating entities

In [ ]:
!bfabric-cli api create --help

```bash
bfabric-cli api create resource name "hello_world_$(date -Iseconds).txt" workunitid 321802 base64 aGVsbG8gd29ybGQ=
```

We can check the result here: https://fgcz-bfabric-test.uzh.ch/bfabric/workunit/show.html?id=321802&tab=resources

Or, we perform a further call.

Since we don't want it to remain there we delete it again: (adjust the ID and **verify** operation before deleting)

```bash
bfabric-cli api delete resource 2805634
```

## Updating entities

The main difference between `bfabric-cli api update` and `bfabric-cli api create` is that `update` requires an id to be passed.
While this distinction is implied in the API the command line tools make this explicit to prevent user errors.

Hence, the `bfabric-cli api update` command in addition to a sequence of attribute value pairs needs to be told about the id of the entity you want to update.

The second difference, is that while create will always be performed without a prompt, the update is considered destructive and hence you will first be presented with a display of the current entity with a prompt to confirm the update.

In [ ]:
!bfabric-cli api update --help

```bash
bfabric-cli api update workunit 321802 description "We are testing some things today"
```

## Deleting entities

We are using yes, so this always deletes the entity for the purpose of the sample, but before doing the deletion you should revise whether it really is the entity that you want to delete.

If you need to perform this operation programmatically, make sure you are deleting **the correct** entities, and then set `--no-confirm` to the command.

In [ ]:
!bfabric-cli api delete --help

# Executable-specific functionality

When working with bfabric apps it can be useful to check the contents of encoded executables, and, upload executables directly.
While originally a XML export is imported with the web app, the CLI supports import of yaml files.

## Show executable

In [ ]:
!bfabric-cli executable show --help

In [ ]:
!bfabric-cli executable show 32859

In [ ]:
!bfabric-cli executable show 33469 

## Upload executable

For B-Fabric application developers you can upload executables from a YAML (this command is experimental):

In [ ]:
!bfabric-cli executable upload --help

# Dataset-specific functionality

Dataset-specific functionality is available through the `bfabric-cli dataset` subcommand.

In [ ]:
!bfabric-cli dataset --help

## Show dataset directly
We can show an existing dataset:

In [ ]:
!bfabric-cli dataset show --help

In [ ]:
!bfabric-cli dataset show 53706

You can also show this as YAML if there is a problem with the rendering (e.g. too many columns for your console):

In [ ]:
!bfabric-cli dataset show 53706 --format yaml

## Download datasets

To download this dataset to a file we can use `bfabric-cli dataset download`:

In [ ]:
!bfabric-cli dataset download --help

In [ ]:
!bfabric-cli dataset download 53706 test_data.parquet --format parquet

In [ ]:
import polars
polars.read_parquet('test_data.parquet')

## Upload datasets

The upload commands are structured into several subcommands, since they do have different parameters between parquet and csv/tsv.

In [ ]:
!bfabric-cli dataset upload --help

In [ ]:
!bfabric-cli dataset upload csv --help

In [ ]:
!bfabric-cli dataset upload parquet --help

To upload the parquet file we just wrote:

In [ ]:
!bfabric-cli dataset upload parquet test_data.parquet --container-id 3000

You should be able to find the `test_data` dataset here now:

https://fgcz-bfabric-test.uzh.ch/bfabric/project/show.html?id=3000&tab=datasets

# Workunit-specific functionality

## Pending workunits

In [ ]:
!bfabric-cli workunit not-available --help

We usually run this find compMS workunits that failed:

```bash
BFABRICPY_CONFIG_ENV=PRODUCTION bfabric-cli workunit not-available
```

## Workunit definition export

When working with bfabric-app-runner it can be useful to export the app definition to a file first.
This functionality is available in `bfabric-cli workunit export-definition` which is very simple to use, by default it will write the `workunit_definition.yml` file if you just give the script a workunit ID.
Of course, you can customize the filename as well.

In [ ]:
!bfabric-cli workunit export-definition --help

In [ ]:
!bfabric-cli workunit export-definition 316119

In [ ]:
!cat workunit_definition.yml

For parsing, a pydantic model is available in the `bfabric` Python package:

In [ ]:
from bfabric.experimental.workunit_definition import WorkunitDefinition
from pathlib import Path
from rich.pretty import pprint
pprint(WorkunitDefinition.from_yaml(Path('workunit_definition.yml')))